<a href="https://colab.research.google.com/github/viksit-siddhant/sentiment_analysis/blob/main/sent_anal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.datasets import imdb
from keras.models import Sequential
import numpy as np
from keras.models import load_model
from keras.preprocessing import sequence

from keras.layers import Dense,Conv1D,GlobalMaxPooling1D,Dropout,Embedding,Activation

In [2]:
#hyperparas
num_of_features = 4096
maxlen = 512

In [19]:
model = Sequential()

model.add(Embedding(num_of_features,64,input_length=maxlen))
model.add(Dropout(0.2))

model.add(Conv1D(128,3,padding='valid',activation='relu',strides=1))
model.add(Conv1D(256,3,padding='valid',activation='relu',strides=2))

model.add(GlobalMaxPooling1D())

model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 512, 64)           262144    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512, 64)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 510, 128)          24704     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 254, 256)          98560     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                

In [21]:
#Run if you want to train your own model
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=num_of_features)

x_train = sequence.pad_sequences(x_train,maxlen=maxlen)
x_test = sequence.pad_sequences(x_test,maxlen=maxlen)

print(x_train.shape,'Shape')

model.fit(x_train,y_train,batch_size=64,epochs=5,validation_data=(x_test,y_test))

(25000, 512) Shape
Epoch 1/5
391/391 [==============================] - 8s 20ms/step - loss: 0.1856 - accuracy: 0.9279 - val_loss: 0.2914 - val_accuracy: 0.8820
Epoch 2/5
391/391 [==============================] - 8s 20ms/step - loss: 0.1279 - accuracy: 0.9537 - val_loss: 0.3491 - val_accuracy: 0.8686
Epoch 3/5
391/391 [==============================] - 8s 21ms/step - loss: 0.0880 - accuracy: 0.9688 - val_loss: 0.3879 - val_accuracy: 0.8763
Epoch 4/5
391/391 [==============================] - 8s 20ms/step - loss: 0.0599 - accuracy: 0.9805 - val_loss: 0.4936 - val_accuracy: 0.8611
Epoch 5/5
391/391 [==============================] - 8s 20ms/step - loss: 0.0444 - accuracy: 0.9846 - val_loss: 0.5018 - val_accuracy: 0.8719


In [14]:
!wget https://www.dropbox.com/s/ldovbkhbv2yavq8/imdb-cnn.h5?dl=0
model = load_model('imdb-cnn.h5?dl=0')

--2020-11-29 13:48:27--  https://www.dropbox.com/s/ldovbkhbv2yavq8/imdb-cnn.h5?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ldovbkhbv2yavq8/imdb-cnn.h5 [following]
--2020-11-29 13:48:27--  https://www.dropbox.com/s/raw/ldovbkhbv2yavq8/imdb-cnn.h5
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbc692297785efe4bffa4047966.dl.dropboxusercontent.com/cd/0/inline/BEHWaemmxzN9_yXHo18CJigOtpqiWMdulB6nhQtUNdL7k-WWsD33H1ungdcryU6DXY-7cwUrKkoV8-DvMVXYNxjwYJW3-WVIyDIX5yVh7IszTw/file# [following]
--2020-11-29 13:48:28--  https://ucbc692297785efe4bffa4047966.dl.dropboxusercontent.com/cd/0/inline/BEHWaemmxzN9_yXHo18CJigOtpqiWMdulB6nhQtUNdL7k-WWsD33H1ungdcryU6DXY-7cwUrKkoV8-DvMVXYNxjwYJW3-WVIyDIX5yVh7IszTw/file
Resolving ucbc692

In [42]:
def preprocess_sentence(sentence,dictionary):
    sentence = sentence.lower()
    x = np.zeros(maxlen)
    punctuation = [':',';','.','\n']
    for char in punctuation:
        sentence.replace(char,'')
    for i,word in enumerate(sentence.split(' ')):
        if word in dictionary and i < maxlen:
            if dictionary[word] <= 4096:
                x[i] = dictionary[word]
    x = x[::-1]
    x = x.reshape((1,-1))
    return x
            
def sent_anal(sentence):
    dictionary = imdb.get_word_index()
    sentence = preprocess_sentence(sentence,dictionary)
    y = model.predict(sentence)
    print(y[0,0])

sent_anal("Terrible")

0.49964264
